In [1]:
!pip install pandas fuzzywuzzy[speedup]

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
from fuzzywuzzy import fuzz, process
from datetime import datetime

In [2]:
import pandas as pd
from fuzzywuzzy import fuzz, process
from datetime import datetime

# 파일 경로 설정
circle_file = 'circle_chart_data_with_dates.csv'  # 서클 차트 파일 경로
youtube_file = 'youtube_artist_weekly_chart_52weeks.csv'  # 유튜브 차트 파일 경로

# 데이터 읽기
circle_data = pd.read_csv(circle_file)
youtube_data = pd.read_csv(youtube_file)

# 가수 이름이 정확히 일치하지 않는 경우 fuzzy matching 적용
def fuzzy_merge(df_1, df_2, key1, key2, threshold=70):
    """
    가수 이름의 유사도 70%를 기준으로 fuzzy matching.
    날짜가 일치하는 경우만 유사도를 확인합니다.
    """
    s = df_2[key2].tolist()

    matches = []
    for index, row in df_1.iterrows():
        date = row['date']
        # 같은 날짜에서만 가수 이름을 매칭
        same_date_youtube = df_2[df_2['Week'] == date]
        if not same_date_youtube.empty:
            match = process.extractOne(row[key1], same_date_youtube[key2].tolist(), scorer=fuzz.token_sort_ratio, score_cutoff=threshold)
            # 만약 매칭되지 않았거나, 유사도가 낮은 경우 포함 여부로 추가 매칭
            if match is None or match[1] < threshold:
                for artist in same_date_youtube[key2]:
                    if row[key1] in artist or artist in row[key1]:
                        match = (artist, 100)  # 포함된 경우 100점으로 설정
                        break
            matches.append(match[0] if match else None)
        else:
            matches.append(None)

    df_1['match'] = matches
    return df_1

# 유튜브 데이터와 서클 차트 데이터를 가수 이름과 날짜로 병합
circle_data = fuzzy_merge(circle_data, youtube_data, 'Artist', 'Artist', threshold=70)

# 날짜가 일치하는 데이터로 필터링 (week와 date가 일치)
merged_data = pd.merge(circle_data, youtube_data, left_on=['match', 'date'], right_on=['Artist', 'Week'], how='left')

# 듀엣 또는 동일 가수가 존재할 경우 더 큰 조회수를 선택
def compare_views(row):
    if pd.notna(row['Weekly Views']) and pd.notna(row['match']):
        solo_views = youtube_data.loc[(youtube_data['Artist'] == row['match']) & (youtube_data['Week'] == row['date']), 'Weekly Views'].max()
        duet_views = row['Weekly Views']
        # 더 큰 조회수 할당
        return max(solo_views, duet_views)
    return row['Weekly Views']

# 더 큰 조회수를 할당하도록 처리
merged_data['Weekly Views'] = merged_data.apply(compare_views, axis=1)

# 필요한 컬럼만 남기고 나머지 제거 (불필요한 열 제거)
final_data = merged_data[['date', 'Rank_x', 'Artist_x', 'Weekly Views']]

# 컬럼 이름 정리
final_data.rename(columns={'Rank_x': 'Rank', 'Artist_x': 'Artist'}, inplace=True)

# 최종 데이터 저장 (CSV 파일로)
final_data.to_csv('circle_chart_with_youtube_views_cleaned.csv', index=False)

print("병합된 데이터가 'circle_chart_with_youtube_views_cleaned.csv' 파일로 저장되었습니다.")


KeyboardInterrupt: 

In [7]:
import pandas as pd

# Load the two uploaded files
circle_chart_df = pd.read_csv('circle_chart_data(20240926)_with_new_artist.csv')
ky_singing_chart_df = pd.read_csv('ky_singing_popular_chart.csv')

# Preview the first few rows of each dataframe to understand their structure
circle_chart_df.head(), ky_singing_chart_df.head()


(         date  Rank                             Title  \
 0  2023-09-24     1                          Love Lee   
 1  2023-09-24     2                            후라이의 꿈   
 2  2023-09-24     3    Smoke (Prod. Dynamicduo, Padi)   
 3  2023-09-24     4                         Super Shy   
 4  2023-09-24     5  Seven (feat. Latto) - Clean Ver.   
 
                        Artist                 Production  Weekly Views  \
 0                   AKMU (악뮤)           YG Entertainment    11929992.0   
 1                   AKMU (악뮤)           YG Entertainment    11929992.0   
 2  다이나믹 듀오 (Dynamic Duo), 이영지  Stone Music Entertainment     4473459.0   
 3                    NewJeans                       ADOR    11655019.0   
 4                          정국               BIGHIT MUSIC     4173726.0   
 
          Genre Runtime     New_Artist  
 0      가요 / 댄스   03:00             악뮤  
 1  가요 / 블루스/포크   03:25             악뮤  
 2    가요 / 랩/힙합   03:30  다이나믹 듀오 , 이영지  
 3      가요 / 댄스   02:35           

In [9]:
# Now perform a left outer join on the two dataframes, matching on date, Title <-> song, and Artist <-> singer
merged_df = pd.merge(circle_chart_df, 
                     ky_singing_chart_df, 
                     how='left', 
                     left_on=['date', 'Title', 'Artist'], 
                     right_on=['date', 'song', 'singer'])


In [10]:
merged_df.head()

,date,Rank,Title,Artist,Production,Weekly Views,Genre,Runtime,New_Artist,rank,song,singer
0,2023-09-24,1,Love Lee,AKMU (악뮤),YG Entertainment,11929992.0,가요 / 댄스,03:00,악뮤,NaN,NaN,NaN
1,2023-09-24,2,후라이의 꿈,AKMU (악뮤),YG Entertainment,11929992.0,가요 / 블루스/포크,03:25,악뮤,NaN,NaN,NaN
2,2023-09-24,3,"Smoke (Prod. Dynamicduo, Padi)","다이나믹 듀오 (Dynamic Duo), 이영지",Stone Music Entertainment,4473459.0,가요 / 랩/힙합,03:30,"다이나믹 듀오 , 이영지",NaN,NaN,NaN
3,2023-09-24,4,Super Shy,NewJeans,ADOR,11655019.0,가요 / 댄스,02:35,뉴진스,NaN,NaN,NaN
4,2023-09-24,5,Seven (feat. Latto) - Clean Ver.,정국,BIGHIT MUSIC,4173726.0,가요 / 댄스,03:04,정국,NaN,NaN,NaN


In [16]:
filtered_df = merged_df[merged_df['rank'].notna()]

In [17]:
filtered_df

,date,Rank,Title,Artist,Production,Weekly Views,Genre,Runtime,New_Artist,rank,song,singer
7,2023-09-24,8,헤어지자 말해요,박재정,로맨틱팩토리,2101694.0,가요 / 발라드,04:04,박재정,1.0,헤어지자 말해요,박재정
9,2023-09-24,10,I AM,IVE (아이브),스타쉽엔터테인먼트,9874062.0,가요 / 댄스,03:04,IVE,68.0,I AM,IVE (아이브)
22,2023-09-24,23,사건의 지평선,윤하,C9엔터테인먼트,4339381.0,가요 / 락,05:01,윤하,53.0,사건의 지평선,윤하
39,2023-09-24,40,주저하는 연인들을 위해,잔나비,페포니 뮤직,3426583.0,가요 / 인디,04:25,잔나비,59.0,주저하는 연인들을 위해,잔나비
51,2023-09-24,52,사랑하지 않아서 그랬어,임한별,플렉스엠,1664288.0,가요 / 발라드,03:44,임한별,70.0,사랑하지 않아서 그랬어,임한별
...,...,...,...,...,...,...,...,...,...,...,...,...
10011,2024-09-08,12,예뻤어,DAY6,JYP Entertainment,17010414.0,가요 / 락,04:43,DAY6,12.0,예뻤어,DAY6
10023,2024-09-08,24,슬픈 초대장,순순희(지환),"제나두엔터테인먼트, 끌림엔터테인먼트",1568194.0,가요 / 발라드,04:07,순순희,4.0,슬픈 초대장,순순희(지환)
10026,2024-09-08,27,비의 랩소디,임재현,"새벽테잎, IK PRODUCTION",NaN,가요 / 발라드,03:55,임재현,24.0,비의 랩소디,임재현
10032,2024-09-08,33,헤어지자 말해요,박재정,로맨틱팩토리,NaN,가요 / 발라드,04:04,박재정,42.0,헤어지자 말해요,박재정


In [18]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10200 entries, 0 to 10199
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          10200 non-null  object 
 1   Rank          10200 non-null  int64  
 2   Title         10200 non-null  object 
 3   Artist        10200 non-null  object 
 4   Production    10200 non-null  object 
 5   Weekly Views  7748 non-null   float64
 6   Genre         10195 non-null  object 
 7   Runtime       10195 non-null  object 
 8   New_Artist    10200 non-null  object 
 9   rank          401 non-null    float64
 10  song          401 non-null    object 
 11  singer        401 non-null    object 
dtypes: float64(2), int64(1), object(9)
memory usage: 956.4+ KB


In [19]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 401 entries, 7 to 10044
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          401 non-null    object 
 1   Rank          401 non-null    int64  
 2   Title         401 non-null    object 
 3   Artist        401 non-null    object 
 4   Production    401 non-null    object 
 5   Weekly Views  269 non-null    float64
 6   Genre         401 non-null    object 
 7   Runtime       401 non-null    object 
 8   New_Artist    401 non-null    object 
 9   rank          401 non-null    float64
 10  song          401 non-null    object 
 11  singer        401 non-null    object 
dtypes: float64(2), int64(1), object(9)
memory usage: 40.7+ KB


In [20]:
merged_df = merged_df.drop(columns=['song', 'singer'])

In [21]:
merged_df.head()

,date,Rank,Title,Artist,Production,Weekly Views,Genre,Runtime,New_Artist,rank
0,2023-09-24,1,Love Lee,AKMU (악뮤),YG Entertainment,11929992.0,가요 / 댄스,03:00,악뮤,NaN
1,2023-09-24,2,후라이의 꿈,AKMU (악뮤),YG Entertainment,11929992.0,가요 / 블루스/포크,03:25,악뮤,NaN
2,2023-09-24,3,"Smoke (Prod. Dynamicduo, Padi)","다이나믹 듀오 (Dynamic Duo), 이영지",Stone Music Entertainment,4473459.0,가요 / 랩/힙합,03:30,"다이나믹 듀오 , 이영지",NaN
3,2023-09-24,4,Super Shy,NewJeans,ADOR,11655019.0,가요 / 댄스,02:35,뉴진스,NaN
4,2023-09-24,5,Seven (feat. Latto) - Clean Ver.,정국,BIGHIT MUSIC,4173726.0,가요 / 댄스,03:04,정국,NaN


In [22]:
# Rename 'rank' to 'ky_rank'
merged_df = merged_df.rename(columns={'rank': 'ky_rank'})

# Replace NaN values with empty strings
merged_df = merged_df.fillna('')

In [23]:
merged_df.head()

,date,Rank,Title,Artist,Production,Weekly Views,Genre,Runtime,New_Artist,ky_rank
0,2023-09-24,1,Love Lee,AKMU (악뮤),YG Entertainment,11929992.0,가요 / 댄스,03:00,악뮤,
1,2023-09-24,2,후라이의 꿈,AKMU (악뮤),YG Entertainment,11929992.0,가요 / 블루스/포크,03:25,악뮤,
2,2023-09-24,3,"Smoke (Prod. Dynamicduo, Padi)","다이나믹 듀오 (Dynamic Duo), 이영지",Stone Music Entertainment,4473459.0,가요 / 랩/힙합,03:30,"다이나믹 듀오 , 이영지",
3,2023-09-24,4,Super Shy,NewJeans,ADOR,11655019.0,가요 / 댄스,02:35,뉴진스,
4,2023-09-24,5,Seven (feat. Latto) - Clean Ver.,정국,BIGHIT MUSIC,4173726.0,가요 / 댄스,03:04,정국,


In [25]:
merged_df.to_csv('circle_chart_data(20240926)_with_new_artist.csv')

In [3]:
score_df = pd.read_csv('finish.csv')
df = pd.read_csv('circle_chart_data(20240927)_jihoon.csv')

In [6]:
import pandas as pd
from fuzzywuzzy import fuzz

# 데이터 로드
score_df = pd.read_csv('finish.csv')
df = pd.read_csv('circle_chart_data(20240927)_jihoon.csv')

# 일치 조건: date가 일치하고 New_Artist가 일치
merged_df = pd.merge(df, score_df, how='left', on=['date', 'New_Artist'])

# Fuzzy Matching 함수 정의 (유사도 60 이상일 경우 매칭)
def fuzzy_match(row, threshold=60):
    if pd.isna(row['Artist']) or pd.isna(row['New_Artist']):
        return False
    # Fuzzy match between Artist and New_Artist
    similarity = fuzz.ratio(str(row['Artist']), str(row['New_Artist']))
    if similarity >= threshold:
        return True
    # Check if score_df's New_Artist is contained within df's Artist
    if str(row['New_Artist']) in str(row['Artist']):
        return True
    return False

# Apply fuzzy matching to the merged dataframe
merged_df['fuzzy_match'] = merged_df.apply(fuzzy_match, axis=1)

# 조건: fuzzy_match가 True인 행과 기존의 매칭된 행들 남기기
final_df = merged_df[merged_df['fuzzy_match'] | pd.notnull(merged_df['Artist'])]

# 매칭되지 않은 score_df 데이터를 남기기
unmatched_score_df = score_df[~score_df['New_Artist'].isin(final_df['New_Artist'])]

# 결과 확인
print(final_df)
print(unmatched_score_df)

# 필요 시 결과를 CSV로 저장
final_df.to_csv('final_merged.csv', index=False)
unmatched_score_df.to_csv('unmatched_score.csv', index=False)


       Unnamed: 0        date  Rank                             Title  \
0               0  2023-09-24     1                          Love Lee   
1               1  2023-09-24     2                            후라이의 꿈   
2               2  2023-09-24     3    Smoke (Prod. Dynamicduo, Padi)   
3               3  2023-09-24     4                         Super Shy   
4               4  2023-09-24     5  Seven (feat. Latto) - Clean Ver.   
...           ...         ...   ...                               ...   
10195       10195  2024-09-08   196                               응급실   
10196       10196  2024-09-08   197                           Run Run   
10197       10197  2024-09-08   198                         Vancouver   
10198       10198  2024-09-08   199                             Lemon   
10199       10199  2024-09-08   200                      첫눈처럼 너에게 가겠다   

                           Artist                              Production  \
0                       AKMU (악뮤)             

In [7]:
final_merged_df = pd.read_csv('final_merged.csv')
unmatched_score_df = pd.read_csv('unmatched_score.csv')

# Perform left outer join on 'date' and 'New_Artist'
final_combined_df = pd.merge(final_merged_df, unmatched_score_df, how='left', on=['date', 'New_Artist'], suffixes=('', '_unmatched'))

In [9]:
final_combined_df = final_combined_df.drop(columns=['Unnamed: 0'])
final_combined_df.head()

,date,Rank,Title,Artist,Production,Weekly Views,Genre,Runtime,New_Artist,ky_rank,ratio,ratio_unmatched
0,2023-09-24,1,Love Lee,AKMU (악뮤),YG Entertainment,11929992.0,가요 / 댄스,3:00,악뮤,NaN,94.32015,NaN
1,2023-09-24,2,후라이의 꿈,AKMU (악뮤),YG Entertainment,11929992.0,가요 / 블루스/포크,3:25,악뮤,NaN,94.32015,NaN
2,2023-09-24,3,"Smoke (Prod. Dynamicduo, Padi)","다이나믹 듀오 (Dynamic Duo), 이영지",Stone Music Entertainment,4473459.0,가요 / 랩/힙합,3:30,"다이나믹 듀오 , 이영지",NaN,NaN,NaN
3,2023-09-24,4,Super Shy,NewJeans,ADOR,11655019.0,가요 / 댄스,2:35,뉴진스,NaN,10.55509,NaN
4,2023-09-24,5,Seven (feat. Latto) - Clean Ver.,정국,BIGHIT MUSIC,4173726.0,가요 / 댄스,3:04,정국,NaN,61.50958,NaN


In [11]:
final_combined_df.to_csv('final_merged_file.csv')

In [12]:
# Fill missing 'ratio' values with 'ratio_unmatched' where applicable
final_combined_df['ratio'] = final_combined_df['ratio'].fillna(final_combined_df['ratio_unmatched'])

# Drop the 'ratio_unmatched' column as it's no longer needed
final_combined_df = final_combined_df.drop(columns=['ratio_unmatched'])

# Save the updated DataFrame to a new CSV file
final_combined_df.to_csv('final_combined_filled.csv', index=False)

# Optionally display the first few rows
print(final_combined_df.head())

         date  Rank                             Title  \
0  2023-09-24     1                          Love Lee   
1  2023-09-24     2                            후라이의 꿈   
2  2023-09-24     3    Smoke (Prod. Dynamicduo, Padi)   
3  2023-09-24     4                         Super Shy   
4  2023-09-24     5  Seven (feat. Latto) - Clean Ver.   

                       Artist                 Production  Weekly Views  \
0                   AKMU (악뮤)           YG Entertainment    11929992.0   
1                   AKMU (악뮤)           YG Entertainment    11929992.0   
2  다이나믹 듀오 (Dynamic Duo), 이영지  Stone Music Entertainment     4473459.0   
3                    NewJeans                       ADOR    11655019.0   
4                          정국               BIGHIT MUSIC     4173726.0   

         Genre Runtime     New_Artist  ky_rank     ratio  
0      가요 / 댄스    3:00             악뮤      NaN  94.32015  
1  가요 / 블루스/포크    3:25             악뮤      NaN  94.32015  
2    가요 / 랩/힙합    3:30  다이나믹 듀오 , 이

In [14]:
import pandas as pd

# Load the two dataframes
circle_chart_df = pd.read_csv('circle_chart_data(20241002)_jihoon.csv')
artist_trends_df = pd.read_csv('artist_trends_32_20241001.csv')

# Perform a left outer join on 'date' and 'Title'
merged_df = pd.merge(circle_chart_df, artist_trends_df[['date', 'Title', 'Score']], how='left', on=['date', 'Title'])

# Save the result to a new CSV file
merged_df.to_csv('circle_chart_with_trends.csv', index=False)

# Optionally display the first few rows of the merged dataframe
print(merged_df.head())

         date  Rank                             Title  \
0  2023-09-24     1                          Love Lee   
1  2023-09-24     2                            후라이의 꿈   
2  2023-09-24     3    Smoke (Prod. Dynamicduo, Padi)   
3  2023-09-24     4                         Super Shy   
4  2023-09-24     5  Seven (feat. Latto) - Clean Ver.   

                       Artist                 Production  Weekly Views  \
0                   AKMU (악뮤)           YG Entertainment    11929992.0   
1                   AKMU (악뮤)           YG Entertainment    11929992.0   
2  다이나믹 듀오 (Dynamic Duo), 이영지  Stone Music Entertainment     4473459.0   
3                    NewJeans                       ADOR    11655019.0   
4                          정국               BIGHIT MUSIC     4173726.0   

         Genre Runtime     New_Artist  ky_rank   n_score  Score  
0      가요 / 댄스    3:00             악뮤      NaN  94.32015  100.0  
1  가요 / 블루스/포크    3:25             악뮤      NaN  94.32015  100.0  
2    가요 / 랩/힙합